In [1]:
# Step to ensure that the venv is being used for the project not local copies, should point at .venv in project.
import sys, shutil
print("python:", sys.executable)
print("uv:", shutil.which("uv")) 

python: c:\repos\Hackathon-2025\.venv\Scripts\python.exe
uv: c:\repos\Hackathon-2025\.venv\Scripts\uv.EXE


In [2]:
# installs into the current Jupyter kernel environment
%pip install -U uv 
#! to run shell commands
!uv pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


Using Python 3.13.7 environment at: C:\repos\Hackathon-2025\.venv
Audited 13 packages in 46ms


In [3]:
# LangChain + MCP Setup for Menu Retrieval (HTTP-based for Jupyter)
import os
from dotenv import load_dotenv
import asyncio
import json
import requests
from typing import Dict, Any, List

# LangChain imports
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

# Official MCP adapter imports for HTTP transport
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

# Load environment variables
load_dotenv()

print("✅ Updated imports with official MCP adapter loaded successfully!")

✅ Updated imports with official MCP adapter loaded successfully!


In [4]:
# MCP Client Setup using Official Adapter with HTTP Transport
import subprocess
import time

# Global MCP client for HTTP
mcp_client = None

async def create_mcp_tools():
    """Create MCP tools using the official LangChain MCP adapter with HTTP transport"""
    global mcp_client
    
    try:
        # Create MultiServerMCPClient with streamable_http transport
        mcp_client = MultiServerMCPClient({
            "menu": {
                "transport": "streamable_http",
                "url": os.getenv("MENU_MCP_URL")
            }
        })
        
        # Get tools from the MCP server
        tools = await mcp_client.get_tools()
        print(f"Loaded {len(tools)} MCP tools: {[tool.name for tool in tools]}")
        return tools
        
    except Exception as e:
        print(f"Error connecting to MCP HTTP server: {e}")
        return []

print("🔗 MCP HTTP adapter setup ready!")


🔗 MCP HTTP adapter setup ready!


In [5]:
# Load MCP Tools using Official Adapter
# The tools will be loaded dynamically when setting up the agent
# No need to manually create tool wrappers - the adapter handles this automatically
print("🛠️ Ready to load MCP tools via official adapter!")


🛠️ Ready to load MCP tools via official adapter!


In [6]:
async def setup_agent():
    """Setup LangChain agent with MCP tools using official adapter"""
    
    # Initialize LLM for Azure OpenAI
    # can get this from Azure Open Ai service -> Azure Ai Foundary Portal
    from langchain_openai import AzureChatOpenAI
    
    llm = AzureChatOpenAI(
        deployment_name=os.getenv("DEPLOYMENT_NAME"),  # Your Azure deployment name
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_version=os.getenv("AZURE_API_VERSION"), 
        temperature=1
    )
    
    # Load MCP tools using official adapter
    tools = await create_mcp_tools()
    
    if not tools:
        print("No MCP tools loaded. Make sure the MCP server is accessible.")
        return None
    
    print(f"Loaded {len(tools)} MCP tools: {[tool.name for tool in tools]}")
    
    # Create system prompt
    system_prompt = """You are an assistant that can help users get information about menu offerings from various restaurants.

    You have access to multiple MCP tools for menu information, including:
    - Searching for menu items by restaurant and cuisine
    - Getting detailed menu item information
    - Assisting with popular menu item recommendations
    - Providing dietary information about menu items
    - Filtering menu items by dietary preferences (e.g., vegan, gluten-free)
    - And more...

    When users ask about menu offerings, use these tools to provide comprehensive information.
    Always be helpful and provide practical advice."""
    
    # Create prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ])
    
    # Create agent
    agent = create_tool_calling_agent(llm, tools, prompt)

    # memory
    memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

    # Create agent executor with tool logging callback and verbose output
    agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)
    
    return agent_executor

# Initialize the agent (now async)
agent_executor = None
print("🤖 Agent setup function ready! Run the next cell to initialize.")


🤖 Agent setup function ready! Run the next cell to initialize.


In [8]:
# Initialize the agent with MCP tools
async def initialize_agent():
    """Initialize the agent with MCP tools"""
    global agent_executor
    print("Initializing agent with MCP tools...")
    agent_executor = await setup_agent()
    if agent_executor:
        print("LangChain agent with MCP tools ready!")
    else:
        print("Failed to initialize agent. Check MCP server connection.")

# Run the initialization
await initialize_agent()


Initializing agent with MCP tools...
Loaded 1 MCP tools: ['get_menu']
Loaded 1 MCP tools: ['get_menu']
LangChain agent with MCP tools ready!


C:\Users\ablackaller\AppData\Local\Temp\ipykernel_54776\402740628.py:51: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [9]:
# User Input Handler + logged agent steps
async def process_user_input(user_input: str) -> str:
    """Process user input and return LLM response using MCP tools"""
    if not agent_executor:
        return "Agent not initialized. Please run the initialization cell first."
    
    try:
        # Use the agent to process the input and get intermediate steps
        result = await agent_executor.ainvoke({"input": user_input})
        output = result.get("output") or result.get("final_output") or ""

        # Print intermediate steps if present
        steps = result.get("intermediate_steps") or []
        for step in steps:
            action = None
            observation = None
            if isinstance(step, tuple) and len(step) == 2:
                action, observation = step
            elif isinstance(step, dict) and "action" in step:
                action = step.get("action")
                observation = step.get("observation")
            else:
                continue

            tool_name = getattr(action, "tool", getattr(action, "tool_name", "unknown"))
            tool_args = getattr(action, "tool_input", getattr(action, "input", None))
            print(f"\n--- Tool: {tool_name}")
            print(f"args: {tool_args}")
            if observation is not None:
                print(f"result: {observation}")
            print("---\n")

        return output
    except Exception as e:
        return f"Error processing request: {str(e)}"

# Interactive function for easy testing
async def ask_assistant(question: str):
    """Easy-to-use function for asking the menu assistant"""
    print(f"🧳 User: {question}")
    print("🤖 Assistant:")
    
    response = await process_user_input(question)
    print(response)
    return response

print("💬 User input handler ready!")

💬 User input handler ready!


In [10]:
# Test MCP server connectivity and tools
async def test_mcp_connection():
    """Test MCP server connection and list available tools"""
    tools = await create_mcp_tools()
    if tools:
        print(f"MCP HTTP server connected successfully!")
        print(f"Available tools: {[tool.name for tool in tools]}")
        for tool in tools:
            print(f"  - {tool.name}: {tool.description}")
    else:
        print("Failed to connect to MCP HTTP server")

# Test MCP HTTP connection
await test_mcp_connection()


Loaded 1 MCP tools: ['get_menu']
MCP HTTP server connected successfully!
Available tools: ['get_menu']
  - get_menu: Get menu information for a specific restaurant

Args:
    restaurantId: The ID of the restaurant

Returns:
    A dictionary containing the menu information or an error message



In [ ]:
# 🚀 EXAMPLE USAGE - Run this cell after setting up your API key!

# Simple question
await ask_assistant("What are all the popular menu items available using restaurantId 2?")
print("\n" + "="*50 + "\n")

# Example 2: Weather + attractions
# await ask_assistant("I'm planning to visit paris tomorrow. What's the weather like and what attractions should I visit?")
# print("\n" + "="*50 + "\n")


🧳 User: What are all the gluten-free menu items available using restaurantId 2?
🤖 Assistant:


> Entering new AgentExecutor chain...

Invoking: `get_menu` with `{'restaurantId': 2}`


{
  "id": 2,
  "name": "Sushi World",
  "items": [
    {
      "name": "California Roll",
      "price": 8.99,
      "size": null,
      "description": "Crab, avocado, and cucumber roll.",
      "is_popular": true,
      "dietary_info": [
        "gluten-free",
        "shellfish"
      ]
    },
    {
      "name": "Spicy Tuna Roll",
      "price": 9.99,
      "size": null,
      "description": "Tuna with spicy mayo.",
      "is_popular": false,
      "dietary_info": null
    },
    {
      "name": "Vegan Avocado Roll (GF)",
      "price": 7.99,
      "size": null,
      "description": "Fresh avocado roll.",
      "is_popular": false,
      "dietary_info": [
        "vegan",
        "gluten-free"
      ]
    },
    {
      "name": "Miso Soup",
      "price": 3.99,
      "size": null,
      "description": 

In [ ]:
# Interactive chat loop — keep asking questions until you exit

# try -
# give me a random attraction
# what about attractions in paris
# more details on eiffel tower
# book eiffel tower


async def chat_loop():
    print("Type 'exit' to quit. Press Enter on an empty line to skip.")
    while True:
        try:
            question = input("You: ").strip()
        except (EOFError, KeyboardInterrupt):
            print("\nExiting.")
            break
        if not question:
            continue
        if question.lower() in ("exit", "quit", "q"):
            print("Goodbye!")
            break
        await ask_assistant(question)

# Start the loop
await chat_loop()


In [ ]:
# Cleanup function for HTTP MCP client
async def cleanup_mcp():
    """Cleanup MCP client and server resources"""
    global mcp_client
    if mcp_client:
        try:
            await mcp_client.close()
            print("MCP client closed")
        except Exception as e:
            print(f"Warning: Error closing MCP client: {e}")
        mcp_client = None

print("🧹 Cleanup function ready!")
